## Using Data From Virtuoso in Tensorflow (SQL Query)

In [52]:
import pandas as pd
import pyodbc
import tensorflow as tf
import datetime
import numpy as np

SHUFFLE_BUFFER = 500
BATCH_SIZE = 2

### Set Connection, and Create Dataframe

In [72]:
cnxn = pyodbc.connect("DSN=Local Virtuoso;UID=dba;pwd=dba")

q = 'SELECT * FROM "tensorflow"."diabetes".data'

d = pd.read_sql_query(q,cnxn)

d.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [14]:
df = pd.DataFrame(d)

outcome = df.pop("Outcome")

df.head()



,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [15]:
feat_names = df.columns.tolist()
feats = df[feat_names]


### Convert to Tensor and continue in TensorFlow

In [16]:
print(tf.convert_to_tensor(feats))

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(feats)
normalizer(feats.iloc[:3])

2021-11-30 11:12:35.345980: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


tf.Tensor(
[[  6.    148.     72.    ...  33.6     0.627  50.   ]
 [  1.     85.     66.    ...  26.6     0.351  31.   ]
 [  8.    183.     64.    ...  23.3     0.672  32.   ]
 ...
 [  2.    122.     70.    ...  36.8     0.34   27.   ]
 [  5.    121.     72.    ...  26.2     0.245  30.   ]
 [  1.    126.     60.    ...  30.1     0.349  47.   ]], shape=(767, 8), dtype=float64)


2021-11-30 11:12:35.708140: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


<tf.Tensor: shape=(3, 8), dtype=float32, numpy=
array([[ 0.6387271 ,  0.84705454,  0.14960383,  0.90778947, -0.6935593 ,
         0.20362139,  0.4676379 ,  1.4246367 ],
       [-0.8458293 , -1.1243613 , -0.16038112,  0.5315605 , -0.6935593 ,
        -0.6842581 , -0.36549374, -0.1917781 ],
       [ 1.2325497 ,  1.9422855 , -0.26370946, -1.2868794 , -0.6935593 ,
        -1.1028302 ,  0.6034746 , -0.10670363]], dtype=float32)>

In [363]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

model = get_basic_model()

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(feats, outcome, epochs=50, batch_size=BATCH_SIZE)

2021-11-30 16:06:48.808011: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-30 16:06:48.808026: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-11-30 16:06:48.808101: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/50
384/384 [==============================] - 0s 638us/step - loss: 0.7308 - accuracy: 0.7223
Epoch 2/50
384/384 [==============================] - 0s 639us/step - loss: 0.4830 - accuracy: 0.7718
Epoch 3/50
384/384 [==============================] - 0s 641us/step - loss: 0.4658 - accuracy: 0.7718
Epoch 4/50
384/384 [==============================] - 0s 647us/step - loss: 0.4555 - accuracy: 0.7823
Epoch 5/50
384/384 [==============================] - 0s 640us/step - loss: 0.4516 - accuracy: 0.7888
Epoch 6/50
384/384 [==============================] - 0s 635us/step - loss: 0.4425 - accuracy: 0.7914
Epoch 7/50
384/384 [==============================] - 0s 638us/step - loss: 0.4330 - accuracy: 0.7901
Epoch 8/50
384/384 [==============================] - 0s 629us/step - loss: 0.4361 - accuracy: 0.7810
Epoch 9/50
384/384 [==============================] - 0s 633us/step - loss: 0.4280 - accuracy: 0.7823
Epoch 10/50
384/384 [==============================] - 0s 627us/step - loss: 0.418

In [112]:
!mkdir -p saved_model
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


## Predictions

In [380]:
sample = [[6,148,72,35,0,33.6,.627,50],[6,148,72,35,0,33.6,.627,60],[1,85,66,29,0,26.6,0.351,55],[1,185,66,29,0,26.6,0,55]]
prediction = model.predict(sample)
print(prediction)

classes = np.argmax(prediction, axis = 1)
print(classes)

[[4.6097789e-02 9.5390218e-01 5.7144423e-09 3.0067941e-09 3.2220500e-09
  1.2087788e-08 2.6904450e-09 6.3051351e-09 3.0234835e-09 5.3067937e-09]
 [2.4759465e-01 7.5240535e-01 1.5435891e-10 5.0920913e-11 5.4625266e-11
  3.3906750e-10 6.5096796e-11 2.5029850e-10 6.3927266e-11 1.1480459e-10]
 [9.9436176e-01 5.6381617e-03 5.9544579e-16 5.5087145e-17 2.5899417e-17
  2.7106052e-16 1.5534199e-16 5.2479398e-16 1.3635084e-16 7.4167740e-17]
 [4.0843606e-01 5.9156394e-01 5.4851596e-15 1.9436704e-15 4.5568208e-15
  1.1111692e-14 4.9686216e-15 2.5152115e-14 4.5871154e-15 7.9486690e-15]]
[1 1 0 1]


## Extra: Prediction via SPARQL query

In [378]:
q3 = '\
SPARQL \
PREFIX : <#> \
SELECT * \
FROM <urn:diabetes:data:test2> \
WHERE \
{\
   ?id :pregnancies ?pregnancies; \
   :glucose ?glucose; :bloodPressure \
   ?bp; :skinThickness ?st; \
   :insulin ?insulin; \
   :bmi ?bmi; \
   :diabetesPedigreeFunction ?dbf; \
   :age ?age\
}'

cursor = cnxn.cursor()

cnxn.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')

cursor.execute(q3)

results = [[]]
rows = cursor.fetchall()

#print(rows[0])

for item in range(1,9) :
    results[0].append(float(rows[1][item]))
print(results)

[[1.0, 85.0, 64.0, 35.0, 0.0, 20.6, 0.0, 25.0]]


In [379]:
prediction2 = model.predict(results)
classes2 = np.argmax(prediction2, axis = 1)
print(classes2)

    

[0]
